<a href="https://colab.research.google.com/github/AliRG97/EconomicComplexityIran/blob/main/Economic_Complexity_2016_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparation of data


we load in the data

In [ ]:
import pandas as pd
import numpy as np

print(2)

df = pd.read_csv("/content/drive/MyDrive/Economics/International trade data 2016 (Atlas of economic complexity)/country_partner_sitcproduct4digit_year_2016.csv",low_memory=False)


2


In [ ]:
columns = ["location_id","product_id","export_value","import_value"]

df_filtered = df[columns]

Filtering the data

In [ ]:
df_grouped = df_filtered.groupby(by=["location_id","product_id"]).agg({"export_value":'sum',"import_value":'sum'})


In [ ]:
df_grouped.reset_index(inplace=True)
df_grouped

,location_id,product_id,export_value,import_value
0,0,651,0,13044
1,0,653,96505,617690
2,0,654,0,5171
3,0,656,129072,21015063
4,0,657,65391,1247522
...,...,...,...,...
150383,250,1432,28367263138,86402925277
150384,250,1433,11255774,791130
150385,250,1434,64563873,20084518
150386,250,1435,3648116,167564


In [ ]:
df_grouped["net_export"] = df_grouped["export_value"] - df_grouped["import_value"]

total_trade = (df_filtered.export_value.sum() + df_filtered.import_value.sum()) / 2.0
country_trade = df_grouped.groupby(by=['location_id']).agg({'export_value':'sum','import_value':'sum'}).reset_index()
country_trade["c_trade"] = (country_trade["export_value"] + country_trade['import_value']) / 2.0
print(country_trade.head())

columns = ["location_id","product_id","net_export"]

df_net = df_grouped[columns]

   location_id  export_value  import_value       c_trade
0            0     300410246    1990968291  1.145689e+09
1            1    1006592402    9310680623  5.158637e+09
2            2   26911666968   11327899365  1.911978e+10
3            3      10586193      61903598  3.624490e+07
4            4    2089667949    4463157535  3.276413e+09


Total trades
Shares
RCA

In [ ]:
df_merge = pd.merge(df_grouped,country_trade,on = ['location_id'])
df_merge.drop(columns=['export_value_y','import_value_y'],axis = 1,inplace = True)
df_merge.rename(columns = {'export_value_x':'export_value','import_value_x':'import_value'},inplace = True)

product_trade = df_grouped.groupby(by=['product_id']).agg({'export_value':'sum','import_value':'sum'}).reset_index()
product_trade['p_trade'] = (product_trade['export_value'] + product_trade['import_value']) / 2.0
product_trade.rename(columns = {'export_value':'e','import_value':'i'},inplace = True)

df_merge = pd.merge(df_merge,product_trade,on = ['product_id'])
df_merge.drop(columns = ['e','i'], inplace = True)

df_merge["RCA"] = (df_merge['export_value'] / df_merge['c_trade']) / (df_merge['p_trade'] / total_trade) # Calculating RCA

# Section A : Calculating Dpp (product proximity matrix)


In [ ]:
n_countries = df_merge['location_id'].nunique()
n_products = df_merge['product_id'].nunique()

mcp = pd.DataFrame(0,index = df_merge['location_id'].unique() ,columns = df_merge['product_id'].unique())
mcp.head()

,651,653,654,656,657,658,659,660,661,662,...,771,788,795,803,838,1269,801,802,819,757
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
for item in range(0,df_merge.shape[0]) :
  if df_merge.iloc[item]['RCA'] >= 1 :
    p_id = df_merge.iloc[item]['product_id']
    c_id = df_merge.iloc[item]['location_id']
    mcp.loc[c_id,p_id] = 1
mcp.head()
products = pd.DataFrame(df_merge['product_id'].unique())
countries = pd.DataFrame(df_merge['location_id'].unique())

In [ ]:
df_iran = df_merge[df_merge.location_id == 107]
df_iran[df_iran.product_id == 701]

,location_id,product_id,export_value,import_value,net_export,c_trade,p_trade,RCA
9299,107,701,334674947,3909120,330765827,5.765837e+10,3.121182e+10,2.867833


extracting kc and kp

In [ ]:
kp = mcp.sum(axis=0)
kc = mcp.sum(axis=1)
print(kp)
kc

651     27
653     29
654     19
656     26
657     24
        ..
1269     7
801      6
802      3
819      1
757      3
Length: 773, dtype: int64


0      29
1      40
2      21
5      15
7      67
       ..
171     3
70      4
36      6
193     0
98      0
Length: 235, dtype: int64

clearing the data from kc = 0

Calculating product distances

In [ ]:
Dpp = mcp.T @ mcp
print(len(products[0]))
kpp = pd.DataFrame(0,index=products[0],columns = products[0])
for i in kpp.columns :
  for j in kpp.index :
    kpp.loc[j,i] = max(kp[i],kp[j])
Dpp = np.divide(Dpp,kpp)
Dpp

773


,651,653,654,656,657,658,659,660,661,662,...,771,788,795,803,838,1269,801,802,819,757
651,1.000000,0.137931,0.111111,0.148148,0.444444,0.111111,0.148148,0.185185,0.074074,0.148148,...,0.148148,0.037037,0.148148,0.037037,0.111111,0.037037,0.037037,0.000000,0.000000,0.000000
653,0.137931,1.000000,0.206897,0.310345,0.172414,0.379310,0.379310,0.310345,0.241379,0.413793,...,0.034483,0.137931,0.103448,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.000000
654,0.111111,0.206897,1.000000,0.307692,0.291667,0.263158,0.304348,0.350000,0.421053,0.307692,...,0.000000,0.052632,0.090909,0.000000,0.105263,0.052632,0.000000,0.000000,0.000000,0.000000
656,0.148148,0.310345,0.307692,1.000000,0.346154,0.307692,0.346154,0.269231,0.538462,0.307692,...,0.192308,0.153846,0.230769,0.038462,0.153846,0.076923,0.038462,0.000000,0.000000,0.000000
657,0.444444,0.172414,0.291667,0.346154,1.000000,0.166667,0.208333,0.333333,0.333333,0.269231,...,0.250000,0.000000,0.208333,0.041667,0.083333,0.000000,0.083333,0.041667,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,0.037037,0.034483,0.052632,0.076923,0.000000,0.200000,0.086957,0.050000,0.157895,0.000000,...,0.000000,0.250000,0.000000,0.142857,0.142857,1.000000,0.142857,0.000000,0.000000,0.142857
801,0.037037,0.034483,0.000000,0.038462,0.083333,0.066667,0.043478,0.150000,0.052632,0.000000,...,0.214286,0.083333,0.136364,0.500000,0.142857,0.142857,1.000000,0.333333,0.000000,0.166667
802,0.000000,0.034483,0.000000,0.000000,0.041667,0.066667,0.000000,0.050000,0.052632,0.000000,...,0.071429,0.000000,0.045455,0.200000,0.000000,0.000000,0.333333,1.000000,0.333333,0.000000
819,0.000000,0.034483,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.052632,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,1.000000,0.000000


Export Dpp

In [ ]:
Dpp.to_csv("/content/drive/MyDrive/Economics/International trade data 2016 (Atlas of economic complexity)/Products_proximity_hs_2016.csv")

In [ ]:
z = kc.loc[kc==0]
print(z)
print(len(kc))
kc.drop(index = z.index,inplace = True)
mcp.drop(index = z.index,inplace = True)
countries.drop(index = z.index, inplace = True)
len(kc)

193    0
98     0
dtype: int64
235


233

calculation Mpp

In [ ]:
products = np.array(products)
countries = np.array(countries)
mpp = pd.DataFrame(0,index = products[1],columns = products[1])
kcd = np.divide([1]*len(countries),kc)
kpd = np.divide([1]*len(products),kp)
kpd = pd.DataFrame(np.diag(kpd))
kcd = pd.DataFrame(np.diag(kcd))
print(kpd.shape)
print(kcd.shape)
print(mcp.shape)
print(pd.DataFrame.transpose(mcp).shape)
kcd.rename(columns = dict(zip(kcd.columns,mcp.index)),index=dict(zip(kcd.index,mcp.index)),inplace = True)
kpd.rename(columns = dict(zip(kpd.columns,mcp.columns)),index = dict(zip(kpd.index,mcp.columns)),inplace = True)
print (mcp.index)
print (kcd.columns)

# Creating matrixes m1 , m2 --- mpp = m1 * m2
m2 = kcd @ mcp
m1 = kpd @ pd.DataFrame.transpose(mcp)
mpp = m1 @ m2
mpp

(773, 773)
(233, 233)
(233, 773)
(773, 233)
Int64Index([  0,   1,   2,   5,   7,   8,   9,  13,  14,  15,
            ...
            225, 241, 218, 198, 229, 233, 105, 171,  70,  36],
           dtype='int64', length=233)
Int64Index([  0,   1,   2,   5,   7,   8,   9,  13,  14,  15,
            ...
            225, 241, 218, 198, 229, 233, 105, 171,  70,  36],
           dtype='int64', length=233)


,651,653,654,656,657,658,659,660,661,662,...,771,788,795,803,838,1269,801,802,819,757
651,0.018382,0.000639,0.002639,0.002010,0.006551,0.000447,0.001693,0.002241,0.000482,0.001278,...,0.002089,0.000146,0.001938,0.000343,0.002099,0.000146,0.000343,0.000000,0.000000,0.000000
653,0.000595,0.006738,0.001006,0.001810,0.000893,0.001526,0.001647,0.001465,0.001051,0.002212,...,0.000192,0.000578,0.001309,0.000308,0.000192,0.000136,0.000192,0.000095,0.000095,0.000000
654,0.003751,0.001536,0.013485,0.003018,0.003487,0.001360,0.001949,0.004001,0.003018,0.003472,...,0.000000,0.000266,0.001674,0.000000,0.002106,0.000266,0.000000,0.000000,0.000000,0.000000
656,0.002088,0.002019,0.002205,0.011977,0.003611,0.001549,0.001886,0.001788,0.004242,0.002415,...,0.002233,0.000763,0.002518,0.000343,0.001447,0.000413,0.000214,0.000000,0.000000,0.000000
657,0.007370,0.001079,0.002760,0.003912,0.022007,0.001061,0.001870,0.003172,0.002582,0.002950,...,0.003112,0.000000,0.002310,0.000386,0.001099,0.000000,0.000794,0.000408,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269,0.000565,0.000565,0.000722,0.001533,0.000000,0.002098,0.003827,0.000812,0.004639,0.000000,...,0.000000,0.002098,0.000000,0.001536,0.000812,0.008417,0.001536,0.000000,0.000000,0.001536
801,0.001543,0.000926,0.000000,0.000926,0.003177,0.000926,0.000926,0.004103,0.000926,0.000000,...,0.004103,0.000926,0.004103,0.005650,0.000926,0.001792,0.011186,0.003949,0.000000,0.001792
802,0.000000,0.000918,0.000000,0.000000,0.003268,0.000918,0.000000,0.003268,0.000918,0.000000,...,0.003268,0.000000,0.003268,0.004630,0.000000,0.000000,0.007898,0.008816,0.000918,0.000000
819,0.000000,0.002755,0.000000,0.000000,0.000000,0.002755,0.000000,0.000000,0.002755,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002755,0.002755,0.000000


Calculation CPI # Seems to be working


In [ ]:
from numpy import linalg as LA
w,v = LA.eig(mpp)
v = np.real(v)
w_index = w.argsort()[-2]
Q = v[:,w_index]
def normalized(vec) :
  vec = vec + np.mean(Q)
  vec = vec / np.std(Q)
  return vec
vec_norm = np.vectorize(normalized)
PCI = vec_norm(Q)
PCI = pd.DataFrame(PCI)
PCI['product_id'] = mpp.index
cov_pci = np.cov(np.array(PCI[[0]]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: Degrees of freedom <= 0 for slice
  
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:2542: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


# Section B: Feasable products

loading in PCI index for 2016 products

In [ ]:
import pandas as pd
import numpy as np

pci = pd.read_csv('/content/drive/MyDrive/Economics/International trade data 2016 (Atlas of economic complexity)/ddf--datapoints--pci--by--product_code_hs--year.txt')
pci.drop(index = pci[pci.year != 2016].index,inplace = True)
pci.drop(columns = 'year', inplace = True)
pci.drop(index = pci[pci.product_code_hs == 'xxxx'].index, inplace = True)
pci.product_code_hs = pci.product_code_hs.astype(int)
pci.rename(columns = {'product_code_hs' : 'product_id'}, inplace = True)
pci_merge = PCI.merge(pci)
cov_m = np.cov(np.array(pci_merge[[0,'pci']]).T)
cov_m


array([[ 1.04553626, -0.10372526],
       [-0.10372526,  0.51676842]])

extracting IRN exported products with RCA > 1

In [ ]:
IRN = 107 # Iran's location_id is 107

IRN_trade = df_merge[df_merge.location_id == IRN]
IRN_p = IRN_trade[IRN_trade.RCA > 1]
IRN_p.reset_index(inplace = True)
IRN_p = IRN_p[['product_id']]
IRN_p

,product_id
0,651
1,668
2,671
3,672
4,693
...,...
75,1153
76,842
77,1341
78,835


adding PCI index

In [ ]:
IRN_p = IRN_p.merge(pci,how = 'inner')
IRN_p

,product_id,pci
0,701,-0.870916
1,706,-0.694442
2,710,-0.773856
3,711,-1.210859
4,712,-1.424108
5,713,-1.671357
6,714,-2.359621
7,902,-1.930218
8,1101,-1.294542
9,1102,-1.033681


# Using Github instead :(

In [ ]:
!pip install ecomplexity
from ecomplexity import ecomplexity
from ecomplexity import proximity

In [ ]:
# importing data
data_url = "https://intl-atlas-downloads.s3.amazonaws.com/country_hsproduct2digit_year.csv.zip"
data = pd.read_csv(data_url, compression="zip", low_memory=False)
data = data[['year','location_code','hs_product_code','export_value']]

# Calculate complexity
trade_cols = {'time':'year', 'loc':'location_code', 'prod':'hs_product_code', 'val':'export_value'}
cdata = ecomplexity(data, trade_cols)

# Calculate proximity matrix
prox_df = proximity(data, trade_cols)

1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016


In [ ]:

prox_df = prox_df[prox_df.year == 2016]
prox_df.reset_index(inplace = True)
prox_df.drop(columns = ['year','index'], inplace = True)
prox_df.to_csv('/content/drive/MyDrive/Economics/International trade data 2016 (Atlas of economic complexity)/Products_proximity_hs_2016.csv')
prox_df

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,hs_product_code_1,hs_product_code_2,proximity
0,01,02,0.333333
1,01,03,0.204082
2,01,04,0.450980
3,01,05,0.377358
4,01,06,0.254902
...,...,...,...
10297,unspecified,99,0.020000
10298,unspecified,financial,0.047619
10299,unspecified,ict,0.014925
10300,unspecified,transport,0.000000
